In [ ]:
from pathlib import Path

from scipy import signal
import matplotlib.pyplot as plt
import numpy as np

import dnb
import brpylib

In [ ]:
filename = '20190122-065346-004'

datafile = f'data/{filename}.ns6'
datapath = Path(datafile)

start_time_s = 1.0
plot_chan = 1

elec_ids     = list(range(1, 97))      # 'all' is default for all (1-indexed)
data_time_s  = 'all' # 2.0 # 'all' # 2.0                     # 'all' is default for all
downsample   = 1        

In [ ]:
# Open file and extract headers
nsx_file = brpylib.NsxFile(str(datapath))

# Extract data - note: data will be returned based on *SORTED* elec_ids, see cont_data['elec_ids']
cont_data = nsx_file.getdata(elec_ids, start_time_s, data_time_s, downsample, full_timestamps=True)

# Close the nsx file now that all data is out
nsx_file.close()

## RAW

In [ ]:
# Plot the data channel
seg_id = 0
ch_idx  = cont_data["elec_ids"].index(plot_chan)
t = cont_data["data_headers"][seg_id]["Timestamp"] / cont_data["samp_per_s"]

In [ ]:
plt.plot(t, cont_data["data"][seg_id][ch_idx])
plt.axis([t[0], t[-1], min(cont_data["data"][seg_id][ch_idx]), max(cont_data["data"][seg_id][ch_idx])])
plt.locator_params(axis="y", nbins=20)
plt.xlabel("Time (s)")
# plt.ylabel("Output (" + nsx_file.extended_headers[hdr_idx]['Units'] + ")")
# plt.title(nsx_file.extended_headers[hdr_idx]['ElectrodeLabel'])
plt.show()

In [ ]:
f0_l = 0.25
f0_h = 4.0
fs = 30000
min_threshold_signal = 90
max_threshold_signal = 250
refractory_period = 0 # unused
delay_to_up_state = 0 # unused
threshold_sinusoid = 100
min_zero_crossing = int(0.25 * fs)
max_zero_crossing = int(1.0 * fs)
logging = False

dnb_filter = dnb.PyFilter(
    f0_l, f0_h, fs, min_threshold_signal, max_threshold_signal,
    refractory_period, delay_to_up_state,
    threshold_sinusoid, min_zero_crossing, max_zero_crossing,
    logging
)

dnb_signal, detected_wave_idx = dnb_filter.filter_signal(cont_data["data"][seg_id][ch_idx])

In [ ]:
plt.plot(t, dnb_signal)
plt.axis([t[0], t[-1], min(dnb_signal), max(dnb_signal)])
plt.locator_params(axis="y", nbins=20)
plt.xlabel("Time (s)")
# plt.ylabel("Output (" + nsx_file.extended_headers[hdr_idx]['Units'] + ")")
# plt.title(nsx_file.extended_headers[hdr_idx]['ElectrodeLabel'])
plt.show()

In [ ]:
len(dnb_signal)

In [ ]:
len(detected_wave_idx)

## Scipy

In [ ]:
b, a = signal.butter(2, [0.25, 4], 'bp', fs=cont_data["samp_per_s"])
scipy_signal = signal.lfilter(b,a,cont_data["data"][seg_id][ch_idx])

In [ ]:
plt.plot(t, scipy_signal)
plt.axis([t[0], t[-1], min(scipy_signal), max(scipy_signal)])
plt.locator_params(axis="y", nbins=20)
plt.xlabel("Time (s)")
# plt.ylabel("Output (" + nsx_file.extended_headers[hdr_idx]['Units'] + ")")
# plt.title(nsx_file.extended_headers[hdr_idx]['ElectrodeLabel'])
plt.show()

In [ ]:
a

In [ ]:
b